In [3]:
!git clone https://github.com/jiegzhan/time-series-forecasting-rnn-tensorflow.git

Cloning into 'time-series-forecasting-rnn-tensorflow'...
remote: Enumerating objects: 78, done.
remote: Total 78 (delta 0), reused 0 (delta 0), pack-reused 78
Unpacking objects: 100% (78/78), done.


In [4]:
!ls

sample_data  time-series-forecasting-rnn-tensorflow


In [5]:
cd time-series-forecasting-rnn-tensorflow/

/content/time-series-forecasting-rnn-tensorflow


In [6]:
!ls

build_model.py	data_helper.py	README.md	      train_predict.py
data		LICENSE		training_config.json


In [7]:
!python train_predict.py

Using TensorFlow backend.
Traceback (most recent call last):
  File "train_predict.py", line 55, in <module>
    train_predict()
  File "train_predict.py", line 13, in train_predict
    train_file = sys.argv[1]
IndexError: list index out of range


In [8]:
!python3 train_predict.py ./data/sales.csv ./training_config.json

Using TensorFlow backend.
/content/time-series-forecasting-rnn-tensorflow/build_model.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(6, 1), return_sequences=True, units=6)`
  model.add(LSTM(input_shape=(layers[1], layers[0]), output_dim=layers[1], return_sequences=True))
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
/content/time-series-forecasting-rnn-tensorflow/build_model.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1)`
  model.add(Dense(output_dim=layers[3]))
Instructions for updating:
Use tf.cast instead.
Train on 20 samples, validate on 3 samples
Epoch 1/100
2019-03-24 13:16:42.292086: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-03-24 13:16:42.292521: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x2e45340 exec

In [10]:
!python3 train_predict.py ./data/daily-minimum-temperatures-in-me.csv ./training_config.json

Using TensorFlow backend.
Traceback (most recent call last):
  File "train_predict.py", line 3, in <module>
    import build_model
  File "/content/time-series-forecasting-rnn-tensorflow/build_model.py", line 2, in <module>
    from keras.layers.core import Dense, Activation, Dropout
  File "/usr/local/lib/python3.6/dist-packages/keras/__init__.py", line 3, in <module>
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/__init__.py", line 6, in <module>
    from . import conv_utils
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/conv_utils.py", line 9, in <module>
    from .. import backend as K
  File "/usr/local/lib/python3.6/dist-packages/keras/backend/__init__.py", line 89, in <module>
    from .tensorflow_backend import *
  File "/usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py", line 5, in <module>
    import tensorflow as tf
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/__init__.py", line 24, in <module>
    from tensorflow

In [13]:
!cat train_predict.py

import sys
import json
import build_model
import data_helper
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def train_predict():
	"""Train and predict time series data"""

	# Load command line arguments 
	train_file = sys.argv[1]
	parameter_file = sys.argv[2]

	# Load training parameters
	params = json.loads(open(parameter_file).read())

	# Load time series dataset, and split it into train and test
	x_train, y_train, x_test, y_test, x_test_raw, y_test_raw,\
		last_window_raw, last_window = data_helper.load_timeseries(train_file, params)

	# Build RNN (LSTM) model
	lstm_layer = [1, params['window_size'], params['hidden_unit'], 1]
	model = build_model.rnn_lstm(lstm_layer, params)

	# Train RNN (LSTM) model with train set
	model.fit(
		x_train,
		y_train,
		batch_size=params['batch_size'],
		epochs=params['epochs'],
		validation_split=params['validation_split'])

	# Check the model against test set
	predicted = build_model.predict_next_timestamp(model, x_test)    